<a href="https://colab.research.google.com/github/Whiteheim/WH/blob/main/Dec14_4_Keras_Callback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 콜백 (Callback)

Model을 학습시킬 때 부가적으로 옵션을 넣어서 수행할 수 있도록 도와주는 객체

모델 학습시 사용하는 fit()에 callbacks란느 파라미터로 지정 가능

In [14]:
import tensorflow as tf
from tensorflow import keras

In [15]:
# keras 내장 dataset에서 mnist 불러오기
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [16]:
# data 정규화 0 ~ 1 사이로
x_train = x_train / x_train.max()
x_test = x_test / 255.

In [17]:
# model 생성
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')

])

In [18]:
# model complie
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 체크포인트 (Model Checkpoint)

epoch(반복) 별로 model의 가중치를 저장하는 역할

Checkpoint Hyper-Parameter
  * filepath : 체크포인트를 저장하는 경로를 지엉
  * sava_weights_only : 가중치만 저장할지 여부 (True, False)
  * save_best_only : monitor 기준 가장 높은 epoch만 저장할지 or epoch마다 저장할지 여부 (True, False)
  * monitor : 저장 시 기준이 되는 지표를 설정
  * verbose : epoch마다 저장 여부를 알려주는 log message 출력 여부 (1(출력), 0(저장))
  

In [19]:
# 체크포인트 설정
checkPoint = keras.callbacks.ModelCheckpoint(
    filepath='checkPoint.ckpt', # ckpt : 모델의 가중치 체크포인트 저장 파일
    # tensorflow 모델 구조 자체는 제외한 가중치'만' 담고 있는 파일
    # 이 파일을 통해 재학습이 가능, 불필요한 정보도 있어서 파일의 크기가 크고 무거움
    save_wights_only=True,# 가중치만 저장
    # False : 모델의 층 (layer), weights 모두 저장
    save_best_only=True,
    # True : monitor 되고 있는 값 기준으로 가장 뛰어난 epoch 모델이 저장
    # False : epoch마다 모델이 filepath{epoch} 형태로 저장
    monitor='val_loss', # model을 저장할 때, 기준이 되는 값을 지정
    # Test_data_set을 기준으로 loss가 가장 적을 때 저장하려면 'val_loss'
    # Trainning_data_set을 기준으로 loss가 가장 적을 때 저장하려면 'loss'
    verbose=1 # 1 : 모델 저장시 문구가 뜸 / 0 : 모델 저장시 문구없이 저장만 됨
)

In [20]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=10,
          callbacks=[checkPoint])

Epoch 1/10
1864/1875 [============================>.] - ETA: 0s - loss: 0.2304 - accuracy: 0.9324
Epoch 1: val_loss improved from inf to 0.10860, saving model to checkPoint.ckpt
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2299 - accuracy: 0.9325 - val_loss: 0.1086 - val_accuracy: 0.9664
Epoch 2/10
1866/1875 [============================>.] - ETA: 0s - loss: 0.0929 - accuracy: 0.9717
Epoch 2: val_loss improved from 0.10860 to 0.10069, saving model to checkPoint.ckpt
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0929 - accuracy: 0.9717 - val_loss: 0.1007 - val_accuracy: 0.9697
Epoch 3/10
1873/1875 [============================>.] - ETA: 0s - loss: 0.0656 - accuracy: 0.9795
Epoch 3: val_loss did not improve from 0.10069
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0656 - accuracy: 0.9795 - val_loss: 0.1099 - val_accuracy: 0.9668
Epoch 4/10
1863/1875 [============================>.] - ETA: 0s - loss: 0.0501 - accuracy: 0.98

In [21]:
# Checkpoint 적용 전 모델
loss, acc = model.evaluate(x_test, y_test)
print('loss : ', loss, 'accuracy : ', acc)

# Checkpoint 적용 후 모델 
model.load_weights('checkPoint.ckpt')
# load_weights를 사용해서 불러오지 않으면, 저장된 가중치는 로드되지 않음 
loss, acc = model.evaluate(x_test, y_test)
print('loss : ', loss, 'accuracy : ', acc)

# 모델 체크포인트에 저장된 가중치를 불러옸을 때는
# 가장 검증손실이 낮았던 epoch 실행 후의 결과가 나옴
# 검증손실이 가장 낮았던 모델 가중치를 사용할 수 있다.

313/313 [==============================] - 1s 2ms/step - loss: 0.1054 - accuracy: 0.9761
loss :  0.10540500283241272 accuracy :  0.9761000275611877
313/313 [==============================] - 1s 2ms/step - loss: 0.0675 - accuracy: 0.9806
loss :  0.06751172244548798 accuracy :  0.9805999994277954
